<a href="https://colab.research.google.com/github/benchoRR/ProductDevelopment/blob/main/AS_RR_Congreso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd
import nltk  # Natural Language Toolkit
import numpy as np
import spacy
import re #regular expressions
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text #para los stop_word


#Spacy , español
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 707.6 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [72]:
#Cargando el archivo con el dataset
dataOrigen=pd.read_csv('IMDB Dataset SPANISH.csv')
columns_remove = ['Unnamed: 0', 'review_en', 'sentiment']
dataOrigen=dataOrigen.drop(columns_remove ,axis=1)
#Remove duplicated
dataOrigen.drop_duplicates(inplace=True)

In [73]:
#Recortando el data set ya que por el tiempo de la presentación no se puede modelar toda la data

# Determinar el tamaño máximo deseado en bytes
max_size = 30 * 1024 * 1024  # 20 MB en bytes

# Calcular el tamaño promedio por fila
row_size = dataOrigen.memory_usage(deep=True).sum() / len(dataOrigen)  # Tamaño promedio por fila en bytes

# Calcular cuántas filas necesitas para alcanzar aproximadamente 20 MB
chunk_size = int(max_size // row_size)  # Número de filas por chunk

# Dividir el DataFrame y guardar las partes
for i in range(0, len(dataOrigen), chunk_size):
    data = dataOrigen.iloc[i:i + chunk_size]

In [74]:
print(data.value_counts('sentimiento'))
print(' ')
print('nulos')
print(data.isnull().sum())

sentimiento
positivo    3734
negativo    3649
Name: count, dtype: int64
 
nulos
review_es      0
sentimiento    0
dtype: int64


In [75]:
data.head()

,review_es,sentimiento
42490,"Vi esta película anoche, una sátira de los pro...",negativo
42491,"Maniratnam, quien en la India, a menudo se com...",negativo
42492,"Freeway Killer, es un loco que dispara a la ge...",negativo
42493,"Película muy lenta, aburrida, enigmática. Tal ...",negativo
42494,Cuéntame como alguien que está feliz de no ver...,positivo


# Limpieza de Datos


In [76]:
def clean_text(text):
    # Convertir el texto a minúsculas
    text = text.lower()

    # Eliminar URLs
    text = re.sub(r'http:?\S+', '', text)

    # Eliminar caracteres de puntuación
    text = re.sub("[%s]" % re.escape(string.punctuation), " ", text)

    # Eliminar caracteres no ASCII, pero incluyendo caracteres del idioma español
    text = re.sub("([^\x00-\x7F\u00C0-\u017F])+", " ", text)

    # Eliminar espacios extra y palabras con menos de 2 caracteres (excepto 'no')
    filter = [palabra for palabra in text.split() if (len(palabra) > 2 or palabra=='no') and palabra.isalpha() ]
    text = " ".join(filter)

    return text

# ejecutra la función , como una función anónima
data.loc[:,'review_es_clean'] = data['review_es'].apply(lambda x: clean_text(x))


<ipython-input-76-9a5dc59aa21f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,'review_es_clean'] = data['review_es'].apply(lambda x: clean_text(x))


# Eliminando StopWords

In [77]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=list(stopwords.words("spanish"))

def remove_stopwords(text):
    stopwords_esp = stopwords.words('spanish')
    words = text.split()
    text= [word for word in words if word not in stopwords_esp]
    return " ".join(text)
data.loc[:,'review_es_clean']=data['review_es_clean'].apply(lambda x: remove_stopwords(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Lematización

In [78]:
nlp = spacy.load('es_core_news_lg')

def lemmatization(text):
  # Procesar el texto de entrada usando el modelo de spaCy
  doc = nlp(text)

  # Obtener la lematización de cada token (palabra) en el documento
  lemas_fila = [token.lemma_ for token in doc]
  text=' '.join(lemas_fila)
  return ' '.join(lemas_fila)
data['review_es_lemma']=data['review_es_clean'].apply(lambda x: lemmatization(x))

<ipython-input-78-46753386d93c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review_es_lemma']=data['review_es_clean'].apply(lambda x: lemmatization(x))


In [79]:
data.head()

,review_es,sentimiento,review_es_clean,review_es_lemma
42490,"Vi esta película anoche, una sátira de los pro...",negativo,película anoche sátira programas estilo realid...,película anoche sátira programa estilo realida...
42491,"Maniratnam, quien en la India, a menudo se com...",negativo,maniratnam india menudo compara prominentes ci...,maniratnam india menudo comparar prominente ci...
42492,"Freeway Killer, es un loco que dispara a la ge...",negativo,freeway killer loco dispara gente autopista mi...,freeway killer loco disparar gente autopista m...
42493,"Película muy lenta, aburrida, enigmática. Tal ...",negativo,película lenta aburrida enigmática tal vez tip...,película lento aburrido enigmático tal vez tip...
42494,Cuéntame como alguien que está feliz de no ver...,positivo,cuéntame alguien feliz ver ningún hastings epi...,cuéntamir alguien feliz ver ninguno hastings e...


In [80]:
columns_remove = ['review_es','review_es_clean']
data=data.drop(columns_remove ,axis=1)

In [81]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data,test_size=0.30,random_state=42)
x_train,y_train=train['review_es_lemma'],train['sentimiento']
x_test,y_test=test['review_es_lemma'],test['sentimiento']
print(len(x_train),len(x_test))

5168 2215




#Vectorizando usando TF-IDF.  (Term Frequency-Inverse Document Frequency)


In [90]:
tfidf=TfidfVectorizer(ngram_range=(1,2),min_df=0.005,max_df=0.70)
X_train=tfidf.fit_transform(x_train).toarray()  #y_train no es necesario porque es la salida
X_test=tfidf.transform(x_test).toarray()  #no toca hacer fit de nuevo ya que tfidf ya hizo fit arriba en train
print("train",X_train.shape)
print("test",X_test.shape)

train (5168, 3553)
test (2215, 3553)


In [91]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [92]:
import keras
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.utils import plot_model
from keras.layers import Convolution1D, AveragePooling1D, Flatten, Dense, Input, MaxPooling1D, Dropout
from sklearn.metrics import classification_report,confusion_matrix

In [93]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)
print(le.classes_)

['negativo' 'positivo']


# Modelo CNN

In [94]:
modelo = Sequential()
modelo.add(Input(shape=(3553,1)))
modelo.add(Convolution1D(128, kernel_size=3, activation="relu"))
modelo.add(MaxPooling1D(pool_size=(2)))
modelo.add(Dropout(0.5))
modelo.add(Convolution1D(32, kernel_size=3, activation="relu"))
modelo.add(MaxPooling1D(pool_size=(2)))
modelo.add(Dropout(0.5))
modelo.add(Flatten())
modelo.add(Dense(32, activation="relu"))
modelo.add(Dense(1, activation="sigmoid"))

modelo.compile(optimizer="adam", loss="binary_crossentropy",
               metrics=["accuracy","Precision"]
)
modelo.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)                    │ (None, 3551, 128)           │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_8 (MaxPooling1D)       │ (None, 1775, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 1775, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_9 (Conv1D)                    │ (None, 1773, 32)            │          12,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_9 (MaxPooling1D)       │ (None, 886, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 886, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 28352)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 32)                  │         907,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 920,161 (3.51 MB)

 Trainable params: 920,161 (3.51 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
modelo.fit(X_train, y_train_encoded, validation_batch_size=(X_test,y_test_encoded),epochs=20)
pred = modelo.predict(X_test)
pred_labels = (pred > 0.5).astype(int)

print(classification_report(y_test_encoded, pred_labels))

Epoch 1/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 93s 559ms/step - Precision: 0.6767 - accuracy: 0.6464 - loss: 0.6293
Epoch 2/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 136s 527ms/step - Precision: 0.8301 - accuracy: 0.8339 - loss: 0.3722
Epoch 3/20
 77/162 ━━━━━━━━━━━━━━━━━━━━ 42s 504ms/step - Precision: 0.8636 - accuracy: 0.8637 - loss: 0.3092

#Pruebas del Modelo

In [63]:
print("Críticas Negativas")
print(modelo.predict(tfidf.transform(['Esta película fue una auténtica pérdida de tiempo. El guion era confuso y la trama carecía de coherencia. Los actores parecían desinteresados y poco comprometidos con sus personajes. Sin duda, una de las peores películas que he visto']).toarray()))
print(modelo.predict(tfidf.transform(['Es increíble que esta película haya recibido buenas críticas. La trama era predecible y los diálogos clichés hicieron que la experiencia fuera monótona. Además, los efectos especiales dejaban mucho que desear y la dirección carecía de originalidad.']).toarray()))
print(modelo.predict(tfidf.transform(['Una película que se presenta como profunda y reflexiva, pero que solo consiguió aburrirme. Los personajes eran insulsos y no despertaron ninguna emoción en mí. No recomendaría perder el tiempo viéndola.']).toarray()))
print(modelo.predict(tfidf.transform(['Esta película prometía ser una emocionante aventura, pero terminó siendo un intento fallido. Aunque los efectos especiales eran impresionantes, no lograron compensar la falta de desarrollo de la trama y la falta de carisma de los personajes. No vale la pena.']).toarray()))
print(modelo.predict(tfidf.transform(['Una comedia que no provocó ni una sola sonrisa en mí. Los chistes eran forzados y los actores parecían sobreactuar en todo momento. Definitivamente, evitaría esta película.']).toarray()))

print("Críticas Positivas")
print(modelo.predict(tfidf.transform(['Esta película es una verdadera obra maestra del cine. El guion es inteligente y conmovedor, manteniéndote cautivado de principio a fin. Las actuaciones son excepcionales, y la dirección es sobresaliente. Sin duda, una película que no debes perderte.']).toarray()))
print(modelo.predict(tfidf.transform(['Una joya del cine. La historia es profundamente emotiva y te hace reflexionar sobre la vida. Los efectos visuales son impresionantes y la banda sonora es cautivadora. Los actores interpretan personajes memorables que te sumergen en la trama.']).toarray()))
print(modelo.predict(tfidf.transform(['Una película que te deja sin aliento. La acción es vertiginosa y las escenas de combate están maravillosamente coreografiadas. Además, el guion, lleno de giros inesperados, te mantiene en vilo hasta el final. Una experiencia cinematográfica inolvidable.']).toarray()))
print(modelo.predict(tfidf.transform(['Una comedia ingeniosa y refrescante que te hará reír a carcajadas. Los diálogos son brillantes y la química entre los actores es perfecta. La dirección logra capturar la esencia de la historia de manera divertida y entretenida. Una película que te dejará de buen humor.']).toarray()))
print(modelo.predict(tfidf.transform(['Una película que te transporta a un asombroso mundo de fantasía. Los efectos especiales son impresionantes y te sumergen en ese universo mágico. La historia es conmovedora y los personajes te roban el corazón. Una película que te hace creer en la magia del cine.']).toarray()))

Críticas Negativas
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
[[0.9621028]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[[0.9793031]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[[0.5029111]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[[0.26314256]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
[[0.56254274]]
Críticas Positivas
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
[[0.70207644]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
[[0.9810323]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[[0.12811089]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[[0.9910828]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[[0.9984299]]


In [ ]:

# Graficar el modelo y guardar la imagen
plot_model(modelo, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

print("Modelo graficado y guardado como 'model_architecture.png'")

Modelo graficado y guardado como 'model_architecture.png'


In [ ]:
#Si el modelo necesita ser exportado.
modelo.save('modelo_imdb.h5')